In [1]:
import pandas as pd
import numpy as py
import os
from bs4 import BeautifulSoup
import json
import requests

## Webscraping artist and what movies/tv show they have acted in

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import time

In [8]:
driver = webdriver.Chrome('./chromedriver')

# list of URLs to scrape

url_ls = ["https://www.imdb.com/name/nm3078932/"]

for url in url_ls:
    driver.get(url)
    
    # Scroll down for elements to populate
    element = driver.find_element("xpath", '''//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[4]/section[2]/div[5]/div/div/div[4]/div/ul/li[14]/div[2]''')
    driver.execute_script("arguments[0].scrollIntoView(true);",element)
    
    # Open up hidden xpaths
    time.sleep(5)
    driver.find_element("xpath", '''//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[4]/section[2]/div[7]/div[1]/div/div[1]/label''').click()
    driver.find_element("xpath", '''//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[4]/section[2]/div[7]/div[2]/div''').click()
    
    # read in html from page
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    soup1 = soup.find_all('div', class_="ipc-accordion__item__content_inner accordion-content")
    
    # Get list of movies the artist acted in
    movie_ls = []
    for i in range(1, 3):
        for x in range(len(soup1[i].find_all('a', class_="ipc-metadata-list-summary-item__t"))):
            movie_ls.append(soup1[i].find_all('a', class_="ipc-metadata-list-summary-item__t")[x].text)
    
    # CLose driver
    driver.close()

movie_ls

C:\Users\Nicholas\AppData\Local\Temp\ipykernel_25580\509655743.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


['Joker: Folie à Deux',
 'House of Gucci',
 'A Star Is Born',
 'American Horror Story',
 'Sin City: A Dame to Kill For',
 'Muppets Most Wanted',
 'Machete Kills',
 'The Simpsons',
 'Men in Black 3',
 'The Lonely Island Feat. Lady Gaga & Justin Timberlake: 3-Way (the Golden Rule)',
 'Gossip Girl',
 "Lady Gaga Feat. Colby O'Donis: Just Dance",
 'The Sopranos']

## Get Relase Date from Google

In [10]:
artist_df = pd.read_csv("ArtistsActorData.csv")
artist_df.head()

,Artist,Movie
0,Lady Gaga,House of Gucci
1,Lady Gaga,A Star Is Born
2,Lady Gaga,American Horror Story
3,Lady Gaga,Sin City: A Dame to Kill For
4,Lady Gaga,Muppets Most Wanted


In [15]:
# Add column to format Google Search
artist_df['Google_Search'] = "release+date+" + artist_df['Movie'].replace(" ", "+", regex=True)
artist_df

,Artist,Movie,Google_Search
0,Lady Gaga,House of Gucci,release+date+House+of+Gucci
1,Lady Gaga,A Star Is Born,release+date+A+Star+Is+Born
2,Lady Gaga,American Horror Story,release+date+American+Horror+Story
3,Lady Gaga,Sin City: A Dame to Kill For,release+date+Sin+City:+A+Dame+to+Kill+For
4,Lady Gaga,Muppets Most Wanted,release+date+Muppets+Most+Wanted
...,...,...,...
68,Justin Timberlake,Edison,release+date+Edison
69,Justin Timberlake,Longshot,release+date+Longshot
70,Justin Timberlake,On the Line,release+date+On+the+Line
71,Justin Timberlake,The Simpsons,release+date+The+Simpsons


In [17]:
search_ls = artist_df['Google_Search'].tolist()
search_ls[:5]

['release+date+House+of+Gucci',
 'release+date+A+Star+Is+Born',
 'release+date+American+Horror+Story',
 'release+date+Sin+City:+A+Dame+to+Kill+For',
 'release+date+Muppets+Most+Wanted']

In [52]:
release_date_ls = []

user_agent = {'User-agent': 'Mozilla/5.0'}

for search in search_ls:
    url = f"https://www.google.com/search?q={search}"
    r = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(r.text,"html.parser")
    search_data1 = soup.find_all("div")
    
    release_date_ls.append(search_data1[35].text)

In [53]:
release_date_ls

['November 24, 2021',
 'October 5, 2018',
 '',
 'August 22, 2014',
 'March 21, 2014',
 'October 11, 2013',
 'December 17, 1989',
 'May 25, 2012',
 '"3-Way (The Golden Rule)" ; Single by the Lonely Island featuring Justin Timberlake and Lady Gaga ; from the album The Wack Album ; Written, May 19, 2011 ; Released\xa0...Background and writing · Composition · Reception · Music video and live...',
 'September 19, 2007',
 '2008',
 'January 10, 1999',
 '2022',
 '2022',
 'October 21, 2022',
 'September 23, 2022',
 '2022',
 '2022',
 '2022',
 'November 5, 2021',
 '2019',
 'Rating  7.9/10  (90) Release date · November 16, 2020 (Italy) · Country of origin. Italy · See more company credits at IMDbPro\xa0...',
 '2019',
 '2019',
 '2019',
 'harrystyles.fandom.com › wiki › BBC_Radio_1_Live_Lounge',
 '2019',
 '2019',
 'Rating  7.4/10  (63) Details · Release date · May 21, 2019 (United States) · Country of origin. Italy · Official site. GUCCI · Language. English · Filming locations · Villa Lante,\xa0...'

In [31]:
search_data1 = soup.find_all("div")

In [49]:
search_data1[35].text

'November 24, 2021'

In [56]:
artist_df['Release_Date'] = release_date_ls

artist_df.head()

,Artist,Movie,Google_Search,Release_Date
0,Lady Gaga,House of Gucci,release+date+House+of+Gucci,"November 24, 2021"
1,Lady Gaga,A Star Is Born,release+date+A+Star+Is+Born,"October 5, 2018"
2,Lady Gaga,American Horror Story,release+date+American+Horror+Story,
3,Lady Gaga,Sin City: A Dame to Kill For,release+date+Sin+City:+A+Dame+to+Kill+For,"August 22, 2014"
4,Lady Gaga,Muppets Most Wanted,release+date+Muppets+Most+Wanted,"March 21, 2014"


In [57]:
artist_df.to_csv("ArtistActor_Export.csv", index=False)